In [1]:
import os
import pandas as pd
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from pmdarima.arima import auto_arima, ndiffs
from pytrends.request import TrendReq

pytrends = TrendReq(hl='NL', tz=360)

C:\Users\31648\AppData\Local\Temp\ipykernel_33760\2383120745.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\31648\AppData\Local\Programs\Python\Python312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
